### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Ce NoteBook est destiné à l'extraction et la visualisation de statistiques relatives aux professeur·e·s et à leurs expertises à partir de l'API de la vitrine de la recherche (SADVR). 
Celles-ci seront intégrées dans un tableau de bord permettant d'avoir un portrait d'ensemble des données.  

---

In [1]:
import pandas as pd
from utils.SADVR_utils import *
import plotly.express as px
import plotly.graph_objects as go

**Chargement des données**

In [2]:
data = updateInfoProfs()

___
**Statistiques sociodémographiques**

- Genre
- Langues parlées
- Langues écrites
- Date d'obtention du dernier diplôme
- Institution de formation / Pays

In [3]:
fonctionsProf = pd.read_csv('utils/fonctionsProfs.csv')['codeSad'].tolist()

demographics = data[['idsadvr', 'sexe', 'langues', 'formations', 'affiliations']]
toNormalize = ['langues', 'affiliations', 'formations', 'formations.disciplines', 'formations.institutions']
for c in toNormalize:
    demographics = explodeNormalize(demographics, c)

columns = pd.read_csv('utils/columnsDemographics.csv')['columns'].tolist()
demographics = demographics[[x for x in demographics.columns if x in columns]]


demographics = demographics[demographics['affiliations.fonction.codeSad'].isin(fonctionsProf)]
demographics.to_csv('tables/demographics.csv', index=False)

nbProfs = len(data.drop_duplicates(subset='idsadvr'))
print(f'Le répertoire compte actuellement {nbProfs} profils de professeur-e-s.')

Le répertoire compte actuellement 2707 profils de professeur-e-s.


In [4]:
demographics

,idsadvr,sexe,langues.nom,langues.medium,affiliations.fonction.codeSad,affiliations.fonction.nom,formations.diplome,formations.annee,formations.institutions.nom,formations.institutions.paysNom,formations.institutions.paysCode
0,in13580,M,NaN,NaN,42001,Professeur titulaire,NaN,NaN,NaN,NaN,NaN
1,in13580,M,NaN,NaN,42032,Professeur associé,NaN,NaN,NaN,NaN,NaN
3,in13581,M,Anglais,Écrit,42001,Professeur titulaire,doctorat IIIème cycle,1984,École Pratique des Hautes Études en Sciences S...,France,FR
4,in13581,M,Anglais,Écrit,42001,Professeur titulaire,doctorat IIIème cycle,1984,École Pratique des Hautes Études en Sciences S...,France,FR
6,in13581,M,Français,Écrit,42001,Professeur titulaire,doctorat IIIème cycle,1984,École Pratique des Hautes Études en Sciences S...,France,FR
...,...,...,...,...,...,...,...,...,...,...,...
11774,in35955,NaN,NaN,NaN,42016,None,NaN,NaN,NaN,NaN,NaN
11775,in35981,M,Français,Écrit,42011,Professeur adjoint,NaN,NaN,NaN,NaN,NaN
11776,in35981,M,Français,Oral,42011,Professeur adjoint,NaN,NaN,NaN,NaN,NaN
11777,in35999,M,NaN,NaN,42011,Professeur adjoint,NaN,NaN,NaN,NaN,NaN


*Genre*

In [1]:
mapping = {'M': 'Hommes', 'F': 'Femmes', 'A': 'Autres'}
genre = plotVariable(demographics, 'sexe', mapping=mapping)

genre

figGenre = px.pie(
    names = genre['labels'],
    values = genre['count'],
    hole=0.5,
    title='Identité de genre',
    width=500,
    #color_discrete_sequence= px.colors.qualitative.Antique   
)

NameError: name 'plotVariable' is not defined

*Langues (parlées, écrites)*

In [6]:
# Langues parlées
langueParle = demographics[demographics['langues.medium'] == 'Oral'].drop(columns=['langues.medium'])
langueParle = pd.DataFrame(plotVariable(langueParle, 'langues.nom'))
langueParle = groupOtherValues(langueParle)

px.pie(
    langueParle,
    values = langueParle['count'], 
    names = langueParle['labels'], 
    title='Langues parlées',
    hole=0.5,
    category_orders={'labels': 
        ['Français', 'Anglais', 'Espagnol; castillan', 
         'Allemand', 'Italien', 'Arabe', 'Autre']},
    color_discrete_sequence= px.colors.qualitative.Pastel1,  
    width=600
)

In [7]:
# Langues écrites

langueEcrite = demographics[demographics['langues.medium'] == 'Écrit'].drop(columns=['langues.medium'])
langueEcrite = pd.DataFrame(plotVariable(langueEcrite, 'langues.nom'))
langueEcrite = groupOtherValues(langueEcrite)

order = [x for x in langueEcrite['labels'] if not (x == 'Autre')] + ['Autre']

px.pie(
    langueEcrite,
    values = langueEcrite['count'], 
    names = langueEcrite['labels'], 
    title='Langues écrites',
    hole=0.5,
    category_orders={'labels': order},
    width=600
)

*Nombre de langues parlées*

In [8]:
langues = demographics[demographics['langues.medium'] == 'Oral'][['idsadvr', 'langues.nom', 'sexe']].drop_duplicates()
nbLangues = pd.DataFrame(langues.groupby(['idsadvr', 'sexe'])['langues.nom'].count()).reset_index().sort_values(by='langues.nom', ascending=False)
freqNbLangues = pd.DataFrame(nbLangues['langues.nom'].value_counts()).reset_index()
freqNbLangues

graphLangueEcrite = px.pie(
    freqNbLangues, 
    values='count', 
    names=freqNbLangues['langues.nom'].apply(lambda x: str(x) + " langue(s) parlée(s)"), 
    title='Nombre de langues parlées',
    hole=0.5,
    color = 'langues.nom',
    width=600
)

graphLangueEcrite.show()

*Fonction*

In [9]:
mappingFonction = pd.read_csv('tables/SADVR_fonctions.csv')[['codeSad', 'nomM']].to_dict('records')
mappingFonction = {x['codeSad'] : x['nomM'] for x in mappingFonction}

fonction = pd.DataFrame(plotVariable(demographics, 'affiliations.fonction.codeSad', mapping=mappingFonction))
fonction

,labels,count
0,Professeur titulaire,879
1,Professeur agrégé,427
2,Professeur associé,321
3,Professeur adjoint,272
4,Professeur adjoint de clinique,268
5,Professeur émérite,150
6,Professeur agrégé de clinique,144
7,Chercheur,78
8,Professeur accrédité,65
9,Professeur titulaire de clinique,42


In [10]:
fig = px.bar(
    fonction.sort_values(ascending=True, by='count'), 
    x="count", 
    y="labels", 
    orientation='h',
    title = 'Fonction',
    height=600)

fig.to_html('visualisations/fonctions.html')
fig.show()

In [12]:
fonctionGenre = demographics[['idsadvr', 'sexe', 'affiliations.fonction.codeSad']].drop_duplicates()
freqFonctionGenre = pd.DataFrame(fonctionGenre[['sexe', 'affiliations.fonction.codeSad']].value_counts()).reset_index()

mapping = pd.read_csv('tables/SADVR_fonctions.csv')[['codeSad', 'nomM']].to_dict('records')
mapping = {x['codeSad'] : x['nomM'] for x in mapping}

freqFonctionGenre['fonction'] = freqFonctionGenre['affiliations.fonction.codeSad'].map(mapping)
freqFonctionGenre.to_csv('tables/statistiques__sociodemographiques/fonctionGenre.csv', index=False)

freqFonctionGenre = freqFonctionGenre[['sexe', 'fonction', 'count']]

fig = go.Figure()

# Femmes
dataF = freqFonctionGenre[freqFonctionGenre['sexe'] == 'F'].sort_values(by='count', ascending=True)


fig.add_trace(go.Bar(
    y= dataF['fonction'],
    x= dataF['count'],
    name='Femmes',
    marker_color='violet',
    orientation = 'h',
))

# Hommes
dataH = freqFonctionGenre[freqFonctionGenre['sexe'] == 'M'].sort_values(by='count', ascending=True)

fig.add_trace(go.Bar(
    y= dataH['fonction'],
    x= dataH['count'],
    name='Hommes',
    marker_color='lightblue',
    orientation = 'h'
))

# Autres
# dataA = freqFonctionGenre[freqFonctionGenre['sexe'] == 'A'].sort_values(by='count', ascending=True)

# fig.add_trace(go.Bar(
#     y= dataA['fonction'],
#     x= dataA['count'],
#     name='Autres',
#     marker_color='brown',
#     orientation = 'h'
# ))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    barmode='group', 
    xaxis_tickangle=-45,
    height=900,
    title = 'Distribution des fonctions par genre')

fig.show()

*Lieu de formation*

In [13]:
paysFormation = pd.DataFrame(plotVariable(demographics, 'formations.institutions.paysNom'))
paysFormation = groupOtherValues(paysFormation, 8)

order = [x for x in paysFormation['labels'] if not (x == 'Autre')] + ['Autre']

px.pie(
    paysFormation, 
    values= paysFormation['count'], 
    names= paysFormation['labels'], 
    title='Formation universitaire Pays',
    hole=0.5,
    width=600, 
    category_orders= {'labels':
        order
    }
)

,labels,count
0,Canada,489
1,États-Unis,175
2,France,111
3,Royaume-Uni,38
4,Suisse,15
5,Belgique,11
6,Pays-Bas,11
7,Allemagne,10
0,Autre,60


In [14]:
order = [x for x in paysFormation['labels'] if not (x == 'Autre')] + ['Autre']

px.pie(
    paysFormation, 
    values= paysFormation['count'], 
    names= paysFormation['labels'], 
    title='Formation universitaire Pays',
    hole=0.5,
    width=600, 
    category_orders= {'labels':
        order
    }
)

*Année d'obtention du dernier diplôme*

In [15]:
# D'abord filtrer pour ne conserver que le dernier diplôme obtenu
anneeDiplome = demographics.sort_values(['idsadvr', 'formations.annee'], ascending=[True, False])
anneeDiplome = anneeDiplome[['idsadvr', 'sexe', 'affiliations.fonction.nom', 'formations.annee']].dropna(subset='formations.annee')
anneeDiplome = anneeDiplome.drop_duplicates(subset=['idsadvr', 'formations.annee'])

anneeDiplomeGenre =  pd.DataFrame(anneeDiplome.drop(columns='idsadvr').value_counts()).reset_index().sort_values(by='formations.annee', ascending=True)

anneeDiplome = pd.DataFrame(plotVariable(anneeDiplome, 'formations.annee'))
anneeDiplome = anneeDiplome.sort_values(by='labels', ascending=True)

In [16]:
fig = px.line(
    anneeDiplome, 
    x=anneeDiplome['labels'], 
    y=anneeDiplome['count'],
    title = "Année d'obtention du dernier diplôme (Ph.D)",
    line_shape = 'spline')
fig.show()

In [17]:
# D'abord filtrer pour ne conserver que le dernier diplôme obtenu
anneeDiplome = demographics.sort_values(['idsadvr', 'formations.annee'], ascending=[True, False])
anneeDiplome = anneeDiplome[['idsadvr', 'sexe', 'formations.annee']].dropna(subset='formations.annee')
anneeDiplome = anneeDiplome.drop_duplicates(subset=['idsadvr', 'formations.annee'])

anneeDiplomeGenre =  pd.DataFrame(anneeDiplome.drop(columns='idsadvr').value_counts()).reset_index().sort_values(by='formations.annee', ascending=True)
anneeDiplomeGenre

,sexe,formations.annee,count
113,M,1959,1
107,M,1961,2
112,M,1962,1
106,M,1963,2
105,M,1964,2
...,...,...,...
72,F,2020,11
81,F,2021,7
102,M,2021,2
101,F,2022,3
